In [1]:
import numpy as np
import scipy
import urllib.request as req
import csv
import codecs
import matplotlib.pyplot as plt

In [2]:
class NeuralNet:
    #initialing
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        # weight matrixes
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
         # learning rate
        self.lr = learningrate
        # using sigmoid function for activation function
        self.activation_function = lambda x: scipy.special.expit(x)
    
    # learn the network
    def train(self, inputs_list, targets_list):
        # make list as 2-dimentional array
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        # calculate signal of hidden section
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate final signals
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        # error levels
        output_errors = targets - final_outputs 
        hidden_errors = np.dot(self.who.T, output_errors)
        
        # learning rate update
        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
    
     # ask to network
    def query(self, inputs_list):
        # make list as 2-dimentional
        inputs = np.array(inputs_list, ndmin=2).T
        
        # calculate hidden section signal
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate final signals
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        return final_outputs

In [3]:
def file_set():
    url = 'https://media.githubusercontent.com/media/freebz/Make-Your-Own-Neural-Network/master/mnist_dataset/mnist_train_100.csv'
    response=req.urlopen(url)
    train_data_file=csv.reader(codecs.iterdecode(response,'utf-8'))
    train_data_list=list(train_data_file)
    response.close()
    url = 'https://media.githubusercontent.com/media/freebz/Make-Your-Own-Neural-Network/master/mnist_dataset/mnist_test_10.csv'
    response = req.urlopen(url)
    test_data_file = csv.reader(codecs.iterdecode(response, 'utf-8'))
    test_data_list = list(test_data_file)
    response.close()
    
    return train_data_list,test_data_list

In [4]:
def teach_network(n,TR_D,output_nodes):
    epochs=5
    for e in range(epochs):
        for record in TR_D:
            all_values = record
            inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
            targets = np.zeros(output_nodes) + 0.01
            targets[int(all_values[0])] = 0.99
            n.train(inputs, targets)
    return n

In [14]:
def test_network(n,TE_D):
    scorecard = []
    
    for record in TE_D:
        all_values = record
        correct_label = int(all_values[0])
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        
        outputs = n.query(inputs)
        label = np.argmax(outputs)
        if (label == correct_label):
            scorecard.append(1)
        else:
            scorecard.append(0)
    scorecard_array = np.asarray(scorecard)
    return scorecard_array

In [6]:
def draw_distrib(scores,rate_scores):
    title='score distribution by hidden layer perceptron number (50-300)'
    plt.scatter(range(50,300),scores[:250],s=3)
    plt.xlabel('perceptrons of hidden layer')
    plt.ylabel('scores')
    plt.title(title)
    plt.savefig(title+'.png')
    plt.show()
    # Image.open(title+'.png').save(title+'.jpg','JPEG')

    title='score distribution by learning rates(0.01-10)'
    plt.plot(np.arange(0.1,10,0.1),rate_scores[:99])
    plt.xlabel('learning_rate')
    plt.ylabel('scores')
    plt.title(title)
    plt.savefig(title+'.png')
    plt.show()
    # Image.open(title+'.png').save(title+'.jpg','JPEG')

In [7]:
def calc_scores(TR_D,TE_D):
    TR_D,TE_D=file_set()
    input_nodes=784
    output_nodes = 10
    
    scores=[]
    learning_rate=0.3 #as constant
    for hidden_nodes in range(50,300):
        n = NeuralNet(input_nodes, hidden_nodes, output_nodes, learning_rate)
        n=teach_network(n,TR_D,output_nodes)
        scorecard=test_network(n,TE_D)
        scorecard_array = np.asarray(scorecard)
        scores.append(float(scorecard_array.sum()) / scorecard_array.size)

    rate_scores=[]
    learning_rate=0.1 #as variables
    hidden_nodes=100 #as constant
    while learning_rate <10:
        n = NeuralNet(input_nodes, hidden_nodes, output_nodes, learning_rate)
        n=teach_network(n,TR_D,output_nodes)
        scorecard=test_network(n,TE_D)

        scorecard_array = np.asarray(scorecard)
        rate_scores.append(float(scorecard_array.sum()) / scorecard_array.size)
        learning_rate+=0.1
        
    return scores,rate_scores

In [10]:
def main():
    input_nodes=784 #27*27pixels
    hidden_nodes = 100
    output_nodes = 10
    learning_rate=0.3
    n = NeuralNet(input_nodes, hidden_nodes, output_nodes, learning_rate)
    TR_D,TE_D=file_set()
    
    n=teach_network(n,TR_D,output_nodes)
    scorecard_array=test_network(n,TE_D)
    
    print(f"performance(IN_node=784 / HI_node=100 / OUT_node=10 / learning_rate=0.3)\n{(float(scorecard_array.sum()) / scorecard_array.size)}")
    
    # HN_sc,R_sc=calc_scores(TR_D,TE_D)
    # draw_distrib(HN_sc,R_sc)

In [13]:
main()

7 7
2 2
1 1
0 0
4 4
1 1
9 4
4 9
4 5
9 9
performance(IN_node=784 / HI_node=100 / OUT_node=10 / learning_rate=0.3)
0.7
